# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [245]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [246]:
weather_info = pd.read_csv("../output_data/cities.csv")
weather_info.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,North Bend,43.4065,-124.2243,42.80,87,90,9.22,US,1613798030
1,Tiksi,71.6872,128.8694,-29.81,85,98,2.55,RU,1613797862
2,East London,-33.0153,27.9116,69.80,88,40,12.66,ZA,1613798037
3,São Filipe,14.8961,-24.4956,69.28,75,11,8.52,CV,1613798041
4,Jamestown,42.0970,-79.2353,18.00,79,90,14.97,US,1613797578


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [247]:
# Configure gmaps
gmaps.configure(api_key=g_key)

weather_info = weather_info.dropna()

# Store latitude and longitude
locations = weather_info[["Lat", "Lng"]].astype(float)

# Store Humidity 
humidity = weather_info["Humidity"].astype(float)

In [248]:
gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

# Display Image
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [249]:
# Narrow cities and drop na's
narrowed_df = weather_info.loc[(weather_info["Max Temp"] < 80) & (weather_info["Max Temp"] > 70) \
                              & (weather_info["Wind Speed"] < 10) \
                              & (weather_info["Cloudiness"] == 0)].dropna()

narrowed_df

# narrowed_df = weather_info

# narrowed_df = narrowed_df.loc[(narrowed_df["Max Temp"] < 80) & (narrowed_df["Max Temp"] > 70)]

# narrowed_df = narrowed_df.loc[narrowed_df["Wind Speed"] < 10]

# narrowed_df = narrowed_df.loc[narrowed_df["Cloudiness"] == 0]

# narrowed_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,Shaowu,27.3409,117.4831,71.06,32,0,0.60,CN,1613798073
46,Te Anau,-45.4167,167.7167,71.89,34,0,6.69,NZ,1613798104
59,Kaeo,-35.1000,173.7833,70.14,58,0,4.65,NZ,1613798128
71,Iralaya,15.0000,-83.2333,76.71,82,0,5.55,HN,1613798147
76,Busselton,-33.6500,115.3333,78.01,49,0,5.01,AU,1613797769
96,Semaria,24.3073,79.9718,78.15,18,0,4.79,IN,1613798206
220,Road Town,18.4167,-64.6167,77.00,86,0,4.00,VG,1613798426
255,Salalah,17.0151,54.0924,73.40,40,0,4.61,OM,1613798555
278,Chicama,-7.8447,-79.1469,72.01,76,0,5.01,PE,1613798586
282,Puerto Colombia,10.9878,-74.9547,78.80,83,0,3.44,CO,1613798587


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [250]:
# hotel_df = weather_info
hotel_df = narrowed_df.loc[:,["City","Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
26,Shaowu,CN,27.3409,117.4831,
46,Te Anau,NZ,-45.4167,167.7167,
59,Kaeo,NZ,-35.1000,173.7833,
71,Iralaya,HN,15.0000,-83.2333,
76,Busselton,AU,-33.6500,115.3333,
96,Semaria,IN,24.3073,79.9718,
220,Road Town,VG,18.4167,-64.6167,
255,Salalah,OM,17.0151,54.0924,
278,Chicama,PE,-7.8447,-79.1469,
282,Puerto Colombia,CO,10.9878,-74.9547,


In [257]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params dictionary to update each iteration
params = {"radius" : 5000,
    "type" : "lodging",
    "keyword" : " Hotel",      
    "key": g_key}

SyntaxError: EOL while scanning string literal (<ipython-input-257-7f6cf6d65d37>, line 5)

In [261]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
  
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}:{city_name}.")
    response = requests.get(base_url, params=params)
    
    response = response.json()

    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
                
    print("------------")

Retrieving Results for Index 26:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 46:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 59:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 71:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 76:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 96:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 220:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 255:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 278:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 282:Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrie

In [262]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,Shaowu,CN,27.3409,117.4831,
46,Te Anau,NZ,-45.4167,167.7167,
59,Kaeo,NZ,-35.1000,173.7833,
71,Iralaya,HN,15.0000,-83.2333,
76,Busselton,AU,-33.6500,115.3333,
96,Semaria,IN,24.3073,79.9718,
220,Road Town,VG,18.4167,-64.6167,
255,Salalah,OM,17.0151,54.0924,
278,Chicama,PE,-7.8447,-79.1469,
282,Puerto Colombia,CO,10.9878,-74.9547,


In [263]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [264]:
# Add marker layer ontop of heat map


# Display figure
